#How To Build a Model: Step By Step


In [2]:
#Pytorch Imports
%load_ext autoreload
%autoreload 2

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.datasets import IMDB
from torchtext.transforms import ToTensor
from torchtext.vocab import build_vocab_from_iterator
#Pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import time




In [3]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from transformers import AutoTokenizer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

C:\Users\haoxu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Pick Your Universal Hyperparamaters:

In [4]:
batch_size = 32
epochs = 10
input_dim = 100
hidden_dim = 50
output_dim = 10
n_filters=30
filter_size=5
drop_frac=0.5
embed_dim = 50
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"==>> torch.cuda.is_available(): {torch.cuda.is_available()}")
print(device)
device = "cpu"

==>> torch.cuda.is_available(): True
cuda


# Setup Your Model Workspace:

Your model architecture, and all of it's relevent code, will go in a folder at the location ./NNDL/(Your Model Name Here)


# Setup Your Dataset/Data Loaders Here

Pytorch usually takes a csv file. You need to write a function( For naming convention let's call it *create_torch_datasets()* ) that takes the universal dataset kyle got from parsing and seperates/loads it into data that you want your model to train on




In [5]:
#Create Custom Dataset Class:
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
from transformers import BertTokenizer
# Load the pre-trained BERT tokenizer
tokenizer = get_tokenizer("basic_english")

# Input text
text = "Hello, how are you doing?"

vec = GloVe(name='6B', dim=embed_dim)
class TextDataset(Dataset):
    def __init__(self, data, root_dir):
        self.root_dir = root_dir
        self.data = data
        self.mapping = {}
        self.cur_map = 0
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        if  type(self.data.iloc[index, 0]) == float:
            t="N/A"
            print('N/A Example  In dataset')
        else:
            t=self.data.iloc[index, 0]
        #print(t)
        tokens = tokenizer.tokenize(t)
        #print(tokens)
        token_length=input_dim
        tokens=tokens+[""] * (token_length-len(tokens))  if len(tokens)<token_length else tokens[:token_length]
        #print(tokens)
        tokens_emb = [vec.stoi.get(token, 0) for token in tokens]
        #print(tokens_emb)
        #print(tokens_emb.shape)
        # print(f"==>> tokens: {tokens}")
        # token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        # print(f"==>> token_ids: {token_ids}")
        
        #input_tensor = torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)
        category_text = os.path.join(self.data.iloc[index, 1])
        if category_text in self.mapping:
            category = self.mapping[category_text]
        else:
            self.mapping[category_text] = self.cur_map
            #print(self.mapping)
            category = self.cur_map
            self.cur_map+=1
        # print(f"self.mapping:{self.mapping}")
        return torch.tensor(tokens_emb),category

In [6]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "Hello, how are you doing?"

# Tokenize the text
tokens = tokenizer.tokenize(text)
tokens_emb = [vec.stoi[token] for token in tokens]
print(torch.tensor(tokens_emb))
# Convert tokens to numerical IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Print the results
print("Original Text:", text)
print("Tokens:", tokens)
print("Token IDs:", torch.tensor(token_ids))

tensor([13075,     1,   197,    32,    81,   914,   188])
Original Text: Hello, how are you doing?
Tokens: ['hello', ',', 'how', 'are', 'you', 'doing', '?']
Token IDs: tensor([7592, 1010, 2129, 2024, 2017, 2725, 1029])


In [7]:
#For Transformer,CNN,RNN
d = pd.read_csv(r'data\products_noimg_uniform_LDKEbb.csv')

text_dataset = TextDataset(data=d,root_dir='/')

train,test,trash = torch.utils.data.random_split(text_dataset,[0.08,0.02,0.9])
train_dataloader =  DataLoader(train, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)
test_dataloader =  DataLoader(test, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)
#vocab= build_vocab_from_iterator(train_dataloader,specials=["<unk>"]).to(device)
#vocab.set_default_index(vocab["<unk>"])
# train_iter= IMDB(split="train")
# test_iter = IMDB(split="test")
# tokenizer = get_tokenizer("basic_english")
# def yield_tokens(data_iter):
#     for _, text in data_iter:
#         yield tokenizer(text)
# train_tokens = []
# vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
# vocab.set_default_index(vocab["<unk>"])
# test_tokens = []
# test_tokens = build_vocab_from_iterator(yield_tokens(test_iter), specials=["<unk>"])



In [8]:
#This is an example of what I'm talking about. The Fashion dataset is pretty easy bc it's function alr exists
# but you need to create your own function to make the training data and test_ data datasets

#For Embedding
text_dataset

Put the dataset into data loader, and check the shape, make sure it's how you want it.


In [9]:
# Create data loaders.
#train_dataloader = DataLoader(text_dataset, batch_size=batch_size,shuffle=True, num_workers=0)
#print(train_dataloader[0])
# for i_batch, sample_batched in enumerate(train_dataloader):
#   print(f"==>> i_batch: {i_batch}")
#   print(f"==>> sample_batched: {len(sample_batched)}")
      #print(i_batch, sample_batched['text'][0][0].item(),
        #sample_batched['category'])

# Display image and label.
#train_features, train_labels = next(iter(train_dataloader))
# print(train_features)
# print(train_labels)
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
# print(f"Label: {label}")

# Setup Model Architecture

1. Create your model architecture in your folder
2. Pick your loss function and optimizer

In [10]:
from NNDL.transformer.architecture import TransformerModel
from NNDL.RNN.architecture import RNN,MyNetwork
from NNDL.Utils.solver import train,test
from NNDL.Utils.weight_tracker import ActivationMonitor
model = TransformerModel(input_dim,embed_dim,output_dim).to(device)
#device="cpu"
#model = RNN(input_dim,hidden_dim,output_dim,embed_dim).to(device)
#model = MyNetwork(embed_dim,n_filters,filter_size,drop_frac,output_dim,embed_dim).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
activation_monitor = ActivationMonitor(model)



# Train the model
This should work to train your model. We may have to make some edits for different optimizations, but we can figure it out.


In [11]:
for t in range(epochs):
    #with torch.autograd.detect_anomaly():
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.368665  [   32/40000]
loss: 1.501646  [ 2080/40000]
loss: 1.209005  [ 4128/40000]
loss: 1.028286  [ 6176/40000]
loss: 1.135450  [ 8224/40000]
loss: 0.906429  [10272/40000]
loss: 0.946409  [12320/40000]
loss: 1.063148  [14368/40000]
loss: 0.898225  [16416/40000]
loss: 0.867113  [18464/40000]
loss: 0.954220  [20512/40000]
loss: 0.449748  [22560/40000]
loss: 1.048797  [24608/40000]
loss: 0.908764  [26656/40000]
loss: 1.008533  [28704/40000]
loss: 0.575284  [30752/40000]
loss: 0.521805  [32800/40000]
loss: 0.874585  [34848/40000]
loss: 0.750549  [36896/40000]
loss: 0.644775  [38944/40000]


: 

In [ ]:
print("Number of layers:", len(activation_monitor.activations))
for i, activation in enumerate(activation_monitor.activations):
    print(f"Layer {i + 1}: {activation.shape}")


Number of layers: 0
